# GASTIADI PROJECT



### Install the required packages


### Imports

In [ ]:
import numpy as np
from numpy.random import RandomState
import pandas as pd
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

### Prep work: Downloading necessary files
Before we get started, we need to store all of the data we'll be using.
* **sentiment500-subset.csv:** cleaned subset of Sentiment1000 data - as positive or negative


import the data in session storage and then copy the path of each data 


In [ ]:
dataset = ('/content/datset_fix.csv')

In [ ]:
dataset= pd.read_csv(dataset)

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
print((dataset.Label == 1).sum()) #urgent
print((dataset.Label == 0).sum()) #unurgent

In [ ]:
dataset.Text

In [ ]:
from collections import Counter

#Count Unique Word

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word]+=1
    return count
  
counter = counter_word(dataset.Text)

In [ ]:
len(counter)

In [ ]:
#Split Dataset Into Training and Validation Set
import numpy
train_size = int(dataset.shape[0]*0.8)

train_ds = dataset[:train_size]
val_ds = dataset[train_size:]

#Split Text and Label
train_sentences = train_ds.Text.to_numpy()
train_labels = train_ds.Label.to_numpy()
val_sentences = val_ds.Text.to_numpy()
val_labels = val_ds.Label.to_numpy()

In [ ]:
train_sentences.shape, val_sentences.shape

In [ ]:
#Tokenizer
from keras.preprocessing.text import Tokenizer

#Vectorize a text corpus by turning each text into a sequence of integers
num_unique_words=len(counter)
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) #fit only to training

In [ ]:
#each word has uniwue index
word_index = tokenizer.word_index

In [ ]:
word_index

In [ ]:
train_sentences=tokenizer.texts_to_sequences(train_sentences)
val_sentences=tokenizer.texts_to_sequences(val_sentences)

In [ ]:
print(train_sentences[10:15])
print(val_sentences[10:15])

In [ ]:
#Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Max number of words in a sequence
max_length = 20

train_padded =pad_sequences(train_sentences, maxlen=max_length, padding='post',truncating='post')
val_padded =pad_sequences(val_sentences, maxlen=max_length, padding='post',truncating='post')
train_padded.shape , val_padded.shape

In [ ]:
#Check the padding
train_padded[10]

In [ ]:
#Create Model
from tensorflow.keras import layers
from tensorflow import keras

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))
model.add(layers.LSTM(64, activation='relu', dropout=0.1))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()


In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.01)
metrics = ['accuracy']

model.compile(loss= loss, optimizer=optim, metrics=metrics)

In [ ]:
history = model.fit(train_padded, train_labels,epochs =20, validation_data=(val_padded,val_labels),verbose=2)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
  
print(plot_history)

In [ ]:
model.evaluate(train_padded)

In [ ]:
predictions = model.predict(train_padded)
predictions = [1 if p> 0.5 else 0 for p in predictions]

In [ ]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

#Fitting model with trainig data
regressor.fit(train_padded, train_labels)

In [ ]:
model.save("gastiadi")

### Predicting with our models

To make a prediction for each of our sentences, you can use `.predict` with each of our models. For example, it would look like this for linear regression:

```python
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
```

To add the prediction for logistic regression, you'd run similar `.predict` code, which will give you a `0` (negative) or a `1` (positive). A difference between the two is that for logistic regression, you can **also ask for the probability that the sentence is in the `1` category** instead of just simply the category. To do that, you use this code:

```python
unknown['pred_logreg_prob'] = linreg.predict_proba(unknown_words_df)[:,1]
```

**Add new columns for each of the models you trained.** If the model has a `.predict_proba`, add that as a column as well. 

* **Tip:** Tab is helpful for knowing whether `.predict_proba` is an option.
* **Tip:** Don't forget the `[:,1]` after `.predict_proba`, it means "give me the probability for category `1`

In [ ]:
# Predict using all our models. 

# Logistic Regression predictions + probabilities
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
unknown['pred_logreg_proba'] = logreg.predict_proba(unknown_words_df)[:,1]

# Random forest predictions + probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['pred_forest_proba'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC predictions
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes predictions + probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['pred_bayes_proba'] = bayes.predict_proba(unknown_words_df)[:,1]

In [ ]:
unknown

## Testing our models

We can actually see **which model performs the best!** Remember how we trained our models on tweets? We can ask each model about each tweet, and see if it gets the right answer.

In [ ]:
dataset.head()

Our original dataframe is a list of many, many tweets. We turned this into `X` - vectorized words - and `y` - whether the tweet is negative or positive.

Before we used `.fit(X, y)` to train on all of our data. Instead, **we can test our models** by doing a test/train split and see if the predictions match the actual labels.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Confusion matrices

To see how well they did, we'll use a ["confusion matrix"](https://en.wikipedia.org/wiki/Confusion_matrix) for each one. I think confusion matrices are called that because they are confusing.

In [ ]:
from sklearn.metrics import confusion_matrix

#### Logistic Regression

In [ ]:
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

#### Random forest

In [ ]:
y_true = y_test
y_pred = forest.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

#### SVC

In [ ]:
y_true = y_test
y_pred = svc.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

#### Multinomial Naive Bayes

In [ ]:
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

### Percentage-based confusion matrices

Those are kind of irritating in that they're just numbers. Let's try percentages instead

#### Logisitic

In [ ]:
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

#### Logistic regression

In [ ]:
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

#### Random forest

In [ ]:
y_true = y_test
y_pred = forest.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

#### SVC

In [ ]:
y_true = y_test
y_pred = svc.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

#### Multinomial Naive Bayes

In [ ]:
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)